version 1.1

# Blackjack OOP

We will begin by defining the classes that will be used in order to separate out different aspects of the game of blackjack. We will model three of the components of the game:

**Card:** A basic playing card. The card belongs to a suit (hearts ♥, diamonds ♦, spades ♠, or clubs ♣) and is worth a certain value.<br>
**Deck:** A collection of cards. The deck shrinks as cards are drawn and contains 52 unique cards.<br>
**Hand:** Each player's assigned cards. A hand is what defines each player's score and thus who wins.

'''
Completed Blackjack program. Source Next Tech sandbox can be found at https://next.tech/projects/94217701d98d/share.
'''

## Get Started

## Create the Card Class

In [5]:
import random # for shuffling

class Card:
    def __init__(self, suit, value):
        self.suit = suit
        self.value = value
# Print out in this format when printing card object
    def __repr__(self):
        return " of ".join((self.value, self.suit))
    def __str__(self):
        return " of ".join((self.value, self.suit))

In [6]:
my_card = Card('diamonds', '2')

In [7]:
print(my_card)

2 of diamonds


## Create the Deck of Cards

We achieve this by using a list comprehension containing lists of every suit and value. We pass each combination over to the initialization for our Card class to create 52 unique Card instances. The Deck will need to be able to be shuffled so that every game is different. We use the shuffle function in the random library to do this for us . To avoid any potential errors, we will only shuffle a deck which still has two or more cards in it, since shuffling one or zero cards is pointless.

In [8]:
class Deck:
    # First create a list comprehension containing lists of every suit and value
    def __init__(self):
        self.cards = [Card(s, v) for s in ["Spades", "Clubs", "Hearts",
                      "Diamonds"] for v in ["A", "2", "3", "4", "5", "6", 
                      "7", "8", "9", "10", "J", "Q", "K"]]

    def shuffle(self):
        # Check if deck has more than one card
        if len(self.cards) > 1:
            random.shuffle(self.cards)

    def deal(self):
        if len(self.cards) > 1:
            return self.cards.pop(0)

In [9]:
deck2 = Deck()
deck2.shuffle()
my_card = deck2.deal()
# print card instant using the card Class __str__.
print(my_card)

10 of Hearts


## Create the Hand Class
A Hand class will need to contain cards just like the Deck class does. It will also be assigned a value by the rules of the game based on which cards it contains. Since the dealer's hand should only display one card, we also keep track of whether the Hand belongs to the dealer to accommodate this rule.
-- If the card's value is numerical, we add its value to the value of this hand (self.value).
-- If it is not numerical, we check to see whether the card is an ace. If it is, we add 11 to the hand's value and set the has_ace flag to True.
-- If it is not an ace, we simply add 10 to the value of the hand.

In [10]:
class Hand:
    def __init__(self, dealer=False):
        self.dealer = dealer # check if this is dealer hand or player (dealer = 'True')
        self.cards = []
        self.value = 0

    def add_card(self, card): # When we call this method we send a card
        self.cards.append(card)

    def calculate_value(self): # We need to calculate the value of each hand
        self.value = 0
        has_ace = False
        for card in self.cards: # check all cards in hand
            if card.value.isnumeric():
                self.value += int(card.value)
            else:               # non numeric  
                if card.value == "A":
                    has_ace = True
                    self.value += 11
                else:
                    self.value += 10
                    
        # Check to see of the Ace brings the hand value above 21 if it does then set to 1 (-10)
        if has_ace and self.value > 21: 
            self.value -= 10

    def get_value(self):
        self.calculate_value()
        return self.value

    def display(self):
        if self.dealer: # If dealer value = True
            print("hidden") # Hide first card
            print(self.cards[1]) #Show second card
        else:
            for card in self.cards:
                print(card)
            print("Value:", self.get_value()) #Get value of two cards


In [11]:
my_deck = Deck()
my_deck.shuffle()
my_hand = Hand()
dealer_hand = Hand()
#Now we add a card to the hand using the add_card() method. uaing as a  parameter the my_deck.deal() method.
my_hand.add_card(my_deck.deal())
my_hand.add_card(my_deck.deal())


dealer_hand.add_card(my_deck.deal())

# Call the display method, which also calculates the hand value
print(my_hand.display())
print(dealer_hand.display())

A of Clubs
10 of Diamonds
Value: 21
None
K of Hearts
Value: 10
None


## Create the Game Class
We will define the game's main loop within its play method, so that to start a game, you will simply need to create an instance of the Game class and call .play() method:

In [26]:
# The game itself is only a loop. No initilasation.
class Game:
    def __init__(self):
        pass

    def play(self):
        playing = True
        # while the playing boolien value is set to True (keep playing)
        while playing:
            self.deck = Deck() # Create a deck
            self.deck.shuffle() # call the shuffle method

            self.player_hand = Hand()   # Create a hand for the player
            self.dealer_hand = Hand(dealer=True) # Create a hand for the dealer

            for i in range(2): #two cards to each hand
                self.player_hand.add_card(self.deck.deal())
                self.dealer_hand.add_card(self.deck.deal())

            print("Your hand is:")
            self.player_hand.display()
            print()
            print("Dealer's hand is:")
            self.dealer_hand.display()

            #Set this variable
            game_over = False

            while not game_over:
                player_has_blackjack, dealer_has_blackjack = self.check_for_blackjack()
                
                if player_has_blackjack or dealer_has_blackjack:
                    game_over = True
                    self.show_blackjack_results(player_has_blackjack, dealer_has_blackjack)
                    continue

                choice = input("Please choose [Hit / Stick] ").lower()
                while choice not in ["h", "s", "hit", "stick"]: #Checks for all instances
                    choice = input("Please enter 'hit' or 'stick' (or H/S) ").lower()
                    
                if choice in ['hit', 'h']: # Deal another hand
                    self.player_hand.add_card(self.deck.deal())
                    self.player_hand.display()
                    if self.player_is_over():
                        print("You have lost!")
                        game_over = True
                        
                else: # Player still in game
                    player_hand_value = self.player_hand.get_value()
                    dealer_hand_value = self.dealer_hand.get_value()

                    print("Final Results")
                    print("Your hand:", player_hand_value)
                    print("Dealer's hand:", dealer_hand_value)

                    if player_hand_value > dealer_hand_value:
                        print("You Win!")
                    elif player_hand_value == dealer_hand_value:
                        print("Tie!")
                    else:
                        print("Dealer Wins!")
                    game_over = True
            
            again = input("Play Again? [Y/N] ")
            while again.lower() not in ["y", "n"]:
                again = input("Please enter Y or N ")
            if again.lower() == "n":
                print("Thanks for playing!")
                playing = False
            else:
                game_over = False

    def player_is_over(self):
        return self.player_hand.get_value() > 21

    def check_for_blackjack(self):
        player = False
        dealer = False
        if self.player_hand.get_value() == 21:
            player = True
        if self.dealer_hand.get_value() == 21:
            dealer = True

        return player, dealer

    def show_blackjack_results(self, player_has_blackjack, dealer_has_blackjack):
        if player_has_blackjack and dealer_has_blackjack:
            print("Both players have blackjack! Draw!")

        elif player_has_blackjack:
            print("You have blackjack! You win!")

        elif dealer_has_blackjack:
            print("Dealer has blackjack! Dealer wins!")

In [27]:
# if __name__ == "__main__":
g = Game()
g.play()

Your hand is:
8 of Clubs
7 of Hearts
Value: 15

Dealer's hand is:
hidden
6 of Spades
Please choose [Hit / Stick] s
Final Results
Your hand: 15
Dealer's hand: 14
You Win!
Play Again? [Y/N] y
Your hand is:
3 of Diamonds
K of Hearts
Value: 13

Dealer's hand is:
hidden
K of Clubs
Please choose [Hit / Stick] h
3 of Diamonds
K of Hearts
7 of Hearts
Value: 20
Please choose [Hit / Stick] s
Final Results
Your hand: 20
Dealer's hand: 17
You Win!
Play Again? [Y/N] n
Thanks for playing!
